# Selection of 50 NLQ queries
 • Model Used : VSLNet (Non-Shared version)

### Mapping Predictions to their Video_Uid
The following blocks ares used to map each predictions to the relative video uid. Somehow, the output of the VSLNet model do not include this information.

In [5]:
import json
import os

# Define paths to data directory and files
annotations_dir = "data/ego4d_data"
predictions_dir = "data/prediction_VSLNet_Non_Shared"

annotations_file = os.path.join(annotations_dir, "v1", "annotations/nlq_val.json")
predictions_file = os.path.join(predictions_dir, "predictions.json")
updated_predictions_file = os.path.join(predictions_dir, "predictions_with_video_id.json")


In [6]:
# Step 1: Create a mapping from clip_uid to video_id
def create_clip_to_video_mapping(annotations_path):
    with open(annotations_path, 'r') as f:
        annotations = json.load(f)
    
    clip_to_video = {}
    for video in annotations["videos"]:
        video_id = video["video_uid"]  # Extract video_id
        for clip in video["clips"]:
            clip_id = clip["clip_uid"]  # Extract clip_id
            clip_to_video[clip_id] = video_id  # Map clip_id to video_id
    
    return clip_to_video

# Step 2: Update predictions with video_id
def add_video_id_to_predictions(predictions_path, clip_to_video_mapping, output_path):
    with open(predictions_path, 'r') as f:
        predictions = json.load(f)
    
    for result in predictions["results"]:
        clip_id = result["clip_uid"]
        result["video_id"] = clip_to_video_mapping.get(clip_id, "unknown")  # Add video_id
    
    # Save updated predictions
    with open(output_path, 'w') as f:
        json.dump(predictions, f, indent=4)

clip_to_video = create_clip_to_video_mapping(annotations_file)
# Update predictions
add_video_id_to_predictions(predictions_file, clip_to_video, updated_predictions_file)    
print(f"Predictions updated and saved to {updated_predictions_file}")


Predictions updated and saved to data/prediction_VSLNet_Non_Shared/predictions_with_video_id.json
